In [254]:
import json
with open("board.json", "r") as f:
    board = json.load(f)
print(f"Loaded {len(board)} actions.")

Loaded 3 actions.


In [255]:
def buildTree(board, getCost, costRemaining):
    return [
        {
            "move": move,
            "costRemaining": costRemaining - getCost(move),
            "nextMoves": buildTree([x for x in board if x != move], getCost, costRemaining - getCost(move)),
        }
        for move in board
        if getCost(move) <= costRemaining
    ]

In [256]:
attackerTree = buildTree(board, lambda x: x["attackerCost"], 10)
defenderTree = buildTree(board, lambda x: x["defenderCost"], 10)

In [257]:
import pygambit

In [258]:
game = pygambit.Game.new_tree()
game.title="subgame"
attacker = game.players.add("Attacker")
defender = game.players.add("Defender")

In [259]:
def labelFor(treeMove):
    return f"{treeMove['attack']['comp']},{treeMove['attack']['vuln']}"

def valueFor(attackerMoves, defenderMoves):
    defenderLabels = [labelFor(m) for m in defenderMoves]
    return sum([
        move["severity"]**2 * move["attackerProb"]
        if labelFor(move) not in defenderLabels else
        move["severity"]**2 * move["attackerProb"] * (1-move["defenderProb"])
        
        for move in attackerMoves
    ])

In [260]:
def addMoves(parent, nextMoves, player, taken, out):
    if len(nextMoves) == 0:
        # outcome = game.outcomes.add("value")
        # outcome[attacker.number] = int(totalValue)
        # outcome[defender.number] = 0
        # parent.outcome = outcome
        out.append({
            "node": parent,
            "taken": taken,
        })
    else:
        gameMove = parent.append_move(player, len(nextMoves))
        for i, treeMove in enumerate(nextMoves):
            gameMove.actions[i].label = labelFor(treeMove["move"])
            addMoves(parent.children[i], treeMove["nextMoves"], player, taken + [treeMove["move"]], out)

In [261]:
out = []
addMoves(game.root, attackerTree, attacker, [], out)
for end in out:
    out2 = []
    addMoves(end["node"], defenderTree, defender, [], out2)
    for final in out2:
        value = valueFor(end["taken"], final["taken"])
        outcome = game.outcomes.add("value")
        outcome[attacker.number] = int(value*10)
        outcome[defender.number] = -int(value*10)
        final["node"].outcome = outcome

C:\Users\TeamD\AppData\Local\Temp\ipykernel_21808\3719867411.py:8: UserWarning: Another outcome with an identical label exists
  outcome = game.outcomes.add("value")


In [262]:
with open("beans.efg","w") as f:
    f.write(game.write())

In [234]:
out[0]["taken"]

[{'attack': {'comp': 0, 'vuln': 0},
  'attackerCost': 4,
  'defenderCost': 4,
  'attackerProb': 0.79,
  'defenderProb': 0.69,
  'severity': 3,
  'risk': 8.82},
 {'attack': {'comp': 0, 'vuln': 1},
  'attackerCost': 4,
  'defenderCost': 4,
  'attackerProb': 0.82,
  'defenderProb': 0.97,
  'severity': 4,
  'risk': 1.18}]